## ![](PySpark.png)

# Apache Hadoop

## Introduction to Hadoop

-   Definition
    -   An open-source software framework for storing and processing large data sets.
-   Components
    -   Hadoop Distributed File System (HDFS): Distributed data storage.
    -   MapReduce: Data processing model.
- Purpose
  - Enables distributed processing of large data sets across clusters of computers.
  
## Hadoop Architecture - HDFS
- HDFS
  - Divides data into blocks and distributes them across different servers for processing.
  - Provides a highly redundant computing environment
    - Allows the application to keep running even if individual servers fail.
    
## Hadoop Architecture- MapReduce

- MapReduce: Distributes the processing of big data files across a large cluster of machines.
  - High performance is achieved by breaking the processing into small units of work that can be run in parallel across nodes in the cluster.
- Map Phase: Filters and sorts data.
  - e.g., Sorting customer orders based on their product IDs, with each group corresponding to a specific product ID.
- Reduce Phase: Summarizes and aggregates results.
  - e.g., Counting the number of orders within each group, thereby determining the frequency of each product ID.

## How Hadoop Works

1. Data Distribution
  - Large data sets are split into smaller blocks.
2. Data Storage
  - Blocks are stored across multiple servers in the cluster.
3. Processing with MapReduce
  - Map Tasks: Executed on servers where data resides, minimizing data movement.
  - Reduce Tasks: Combine results from map tasks to produce final output.
4. Fault Tolerance
  - Data replication ensures processing continues even if servers fail.
  
## Extending Hadoop for Real-Time Processing

- Limitation of Hadoop
  - Hadoop is originally designed for batch processing.
    - Batch Processing: Data or tasks are collected over a period of time and then processed all at once, typically at scheduled times or during periods of low activity.
    - Results come after the entire dataset is analyzed.
- Real-Time Processing Limitation:
  - Hadoop cannot natively process real-time streaming data (e.g., stock prices flowing into stock exchanges, live sensor data)
- Extending Hadoop’s Capabilities
  - Both Apache Storm and Apache Spark can run on top of Hadoop clusters, utilizing HDFS for storage.

# Apache Spark

## Spark

- Apache Spark: distributed processing system used for big data workloads. a unified computing engine and computer clusters
  - It contains a set of libraries for parallel processing for data analysis, machine learning, graph analysis, and streaming live data.
  
## Spark Application Structure on a Cluster of Computers

- Driver Process
  - Communicates with the cluster manager to acquire worker nodes.
  - Breaks the application into smaller tasks if resources are allocated.
- Cluster Manager
  - Decides if Spark can use cluster resources (machines/nodes).
  - Allocates necessary nodes to Spark applications.
- Worker Nodes
  - Execute tasks assigned by the driver program.
  - Send results back to the driver after execution.
  - Can communicate with each other if needed during task execution.

# Spark vs. Hadoop

## Hadoop MapReduce: The Challenge
- Sequential Multi-Step Process:
  - Reads data from the cluster.
  - Processes data.
  - Writes results back to HDFS.
- Disk Input/Output Latency:
  - Each step requires disk read/write.
  - Results in slower performance due to latency.

## Apache Spark: The Solution
- In-Memory Processing:
  - Loads data into memory once.
  - Performs all operations in-memory.
- Data Reuse:
  - Caches data for reuse in multiple operations (ideal for iterative tasks like machine learning).
- Faster Execution:
  - Eliminates multiple disk I/O steps.
  - Dramatically reduces latency for interactive analytics and real-time processing.

# PySpark on Google Colab
## PySpark = Spark + Python

- pyspark is a Python API to Apache Spark.
  - API: application programming interface, the set of functions, classes, and variables provided for you to interact with.
  - Spark itself is coded in a different programming language, called Scala.
- We can combine Python, pandas, and PySpark in one program.
  - Koalas (now called pyspark.pandas) provides a pandas-like porcelain on top of PySpark.
  
## Spark DataFrame vs. Pandas DataFrame

-  What makes a Spark DataFrame different from other DataFrame?
  - Spark DataFrames are designed for big data and distributed computing.
- Spark DataFrame:
  - Data is distributed across a cluster of machines.
  - Operations are executed in parallel on multiple nodes.
  - Can process datasets that exceed the memory of a single machine.
- Other DataFrames (e.g., Pandas):
  - Operate on a single machine.
  - Entire dataset must fit into memory.
  - Limited by local system resources.
  
## Lazy Evalutation and Optimization
- Spark DataFrame:
  - Uses lazy evaluation: transformations are not computed until an action is called.
  - Optimize query execution.
- Other DataFrames:
  - Operations are evaluated eagerly (immediately).
  - No built-in query optimization across multiple operations.

## Scalability
- Spark DataFrame:
  - Designed to scale to petabytes of data.
  - Utilizes distributed storage and computing resources.
  - Ideal for large-scale data processing and analytics.
- Other DataFrames:
  - Best suited for small to medium datasets.
  - Limited by the hardware of a single computer.
  
## Fault Tolerance
- Spark DataFrame:
  - Built on Resilient Distributed Datasets (RDDs).
  - Automatically recovers lost data if a node fails.
  - Ensures high reliability in distributed environments.
- Other DataFrames:
  - Typically lack built-in fault tolerance.
  - Failures on a single machine can result in data loss.
  
# PySpark Basics

## The SparkSession Entry Point

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

## Reading a CSV file into the Spark Framework


In [ ]:
path = 'https://bcdanl.github.io/data/df.csv'
df = spark.read.csv(path, 
                    inferSchema=True,
                    header=True)
df.show()

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
df_pd = pd.read_csv('https://bcdanl.github.io/data/nba.csv')
df = spark.createDataFrame(df_pd)

## Getting a Summary of Data
- df.printSchema(): prints the schema (column names and data types).
- df.columns: returns the list of columns.
- df.dtypes: returns a list of tuples (columnName, dataType).
- df.count(): returns the total number of rows.
- df.describe(): returns basic statistics of numerical/string columns (mean, count, std, min, max).


In [ ]:
df.describe().show()

## Selecting Columns

In [ ]:
# Single column -> returns a DataFrame with one column
df.select("Name").show(5)

# Multiple columns -> pass a list-like of column names
df.select("Name", "Team", "Salary").show(5)

## Counting Methods

In [ ]:
# Counting how many total rows
nba_count = df.count()

# Count distinct values in one column
from pyspark.sql.functions import countDistinct
num_teams = df.select(countDistinct("Team")).collect()[0][0]

# GroupBy a column and count occurrences
df.groupBy("Team").count().show(5)

## Sorting Methods

In [ ]:
# Sort by a single column ascending
df.orderBy("Name").show(5)

# Sort by descending
from pyspark.sql.functions import desc
df.orderBy(desc("Salary")).show(5)

# Sort by multiple columns
df.orderBy(["Team", desc("Salary")]).show(5)

- Sorting by One or More Variables


In [ ]:
# nsmallest example:
df.orderBy("Salary").limit(5).show()

# nlargest example:
df.orderBy(desc("Salary")).limit(5).show()

- Equivalent of Pandas nsmallest or nlargest

## Indexing and Row Access

In [ ]:
# Example: filter by condition
df.filter("Team == 'New York Knicks'").show()
df.limit(5).show()
df.take(5)
df.collect()

## Adding, Removing, Renaming, and Relocating Variables


In [ ]:
# Add a column "Salary_k" using a column expression col()
df = df.withColumn("Salary_k", col("Salary") / 1000) 

- Adding Columns with withColumn()


In [ ]:
df = df.drop("Salary_k")  # remove a single column
df = df.drop("Salary_2x", "Salary_3x")  # remove multiple columns

- Removing Columns with drop()


In [ ]:
df = df.withColumnRenamed("Birthday", "DateOfBirth")

- Renaming Columns with withColumnRenamed()


In [ ]:
df = df.select("Name", "Team", "Position", "Salary", "DateOfBirth")

- Rearranging Columns

## Mathematical & Vectorized Operations


In [ ]:
# Summaries for numeric columns
df.selectExpr(
    "mean(Salary) as mean_salary",
    "min(Salary) as min_salary",
    "max(Salary) as max_salary",
    "stddev_pop(Salary) as std_salary"
).show()

In [ ]:
from pyspark.sql import functions as F
# Pre-compute the average salary (pulls it back as a Python float)
salary_mean = df.select(F.avg("Salary").alias("mean_salary")).collect()[0]["mean_salary"]

df2 = (
    df
    .withColumn("Salary_2x", F.col("Salary") * 2)    # Add Salary_2x
    .withColumn(
        "Name_w_Position",           # Concatenate Name and Position
        F.concat(F.col("Name"), F.lit(" ("), F.col("Position"), F.lit(")")))
    .withColumn(
        "Salary_minus_Mean",        # Subtract mean salary
        F.col("Salary") - F.lit(salary_mean))
)

## Filtering by a Condition

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
df_pd = pd.read_csv("https://bcdanl.github.io/data/employment.csv")
df_pd = df_pd.where(pd.notnull(df_pd), None)  # Convert NaN to None
df = spark.createDataFrame(df_pd)

df.filter(col("Salary") > 100000).show()

In [ ]:
df.filter(
    ( col("Team") == "Finance" ) & 
    ( col("Salary") >= 100000 )
).show()

In [ ]:
df.filter(
    (col("Team") == "Finance") | 
    (col("Team") == "Legal")   | 
    (col("Team") == "Sales")
).show()